In [68]:
# !pip install openai

In [101]:
import pandas as pd
import numpy as np
import re
import json

from sklearn.metrics import (
    accuracy_score, 
    f1_score, 
    recall_score,
    precision_score,
    roc_auc_score, confusion_matrix, roc_curve, classification_report
)

import os
from openai import OpenAI
from tqdm.notebook import tqdm

In [82]:
with open('../secrets.json') as f:
    secrets = json.load(f)

In [83]:
client = OpenAI(api_key=secrets['chatgtp_api_key_2'],)

# Extract one of the folds

In [84]:
with open('../datasets/used_data/03_bert_like_models/02_topics.npy', 'rb') as f:
    topics = np.load(f).tolist()

y_train_u_topics = pd.DataFrame()
y_train_u_topics['topic'] = topics
y_train_u_topics['n'] = 1

In [85]:
cv_fold = []

for i in y_train_u_topics['topic'].unique().reshape(10,-1):
    train_cv = y_train_u_topics.index[ ~np.isin(y_train_u_topics["topic"], i) ].values
    test_cv = y_train_u_topics.index[ np.isin(y_train_u_topics["topic"], i) ].values
    
    cv_fold.append( [train_cv, test_cv])

In [86]:
train_i_chat, test_i_chat = cv_fold[0]

In [87]:
df_basic_info = pd.read_parquet('../datasets/used_data/03_bert_like_models/01_basic_info_data.parquet')

In [106]:
df_basic_info.head()

,assestment,text,text_clean
0,0,(...) udzielenie bankowi centralnemu Ukrainy t...,udzielenie bankowi centralnemu Ukrainy tzw. s...
1,0,Polska już w grudniu (2021 roku – przyp. Dema...,Polska już w grudniu 2021 roku miała trzecią n...
2,0,(Bartosz Cichocki – przyp. Demagog) był jedyny...,Bartosz Cichocki był jedynym ambasadorem z kra...
3,1,Będąc (prezes NBP Adam Glapiński – przyp. D...,Będąc prezes NBP Adam Glapiński przez 6 lat os...
4,0,W latach 2018–2019 Narodowy Bank Polski zakupi...,W latach 2018–2019 Narodowy Bank Polski zakupi...


# Get results from ChatGPT api

In [99]:
messages = [ {"role": "system", "content": "You are a intelligent assistant."} ]

promopt = 'Zaklasyfikuj poniższe zdanie jako prawdziwe (True) lub fałszywe (Fake), odpowiedz uzywajac jednego słow - Fake/True:'

In [98]:
# pattern = r'statement:\s*(True|Fake)'

y_zero_shot_test = []

In [104]:
# for i, t in tqdm(df_basic_info.loc[test_i_chat, :]['text'].items(), total=test_i_chat.shape[0]):   
#     completion = client.chat.completions.create(
#         messages=[
#             {
#                 "role": "system",
#                 "content": f'{promopt} {t}',
#             }
#         ],
#         model="gpt-3.5-turbo",
#     )

#     reply = completion.choices[0].message.content
    
#     # match = re.search(pattern, reply)
#     # result = match.group(1)

#     y_zero_shot_test.append( 'True' in reply )

In [105]:
# y_zero_shot_test = []

# pattern = r'statement:\s*(True|Fake)'

# for r in results_chatgpt:
#     match = re.search(pattern, r)
#     result = match.group(1)
    
#     y_zero_shot_test.append( result == 'True' )

In [26]:
len(results_chatgpt)

24

In [37]:
y_test = df_basic_info.loc[test_i_chat, 'assestment'][:24].values == np.array(y_zero_shot_test)

In [38]:
y_pred = np.array(y_zero_shot_test)

In [39]:
accuracy_score(y_test, y_pred) 

0.2916666666666667

In [40]:
f1_score(y_test, y_pred) 

0.32